In [1]:
from typing import Optional
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
data_dir = Path("/workspaces/pyplier") / "tests" / "data" / "solveU"

In [3]:
Z = pd.read_csv(data_dir / "Z.csv.gz", index_col="gene")
Chat = pd.read_csv(data_dir / "Chat.csv.gz", index_col="pathway")
priorMat = pd.read_csv(data_dir / "priorMat.csv.gz", index_col="gene")
penalty_factor = pd.read_csv(data_dir / "penalty_factor.csv")
pathwaySelection = "complete"
glm_alpha = 0.9
maxPath = 10
target_frac = 0.7
L3 = None

In [4]:
Ur = Chat @ Z

In [5]:
Ur

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.002890,-0.000976,-0.016588,0.001448,0.018986,-0.005619,0.012043,0.004185,0.004563,-0.017232,...,0.031321,0.004945,-0.001207,-0.010078,-0.006603,-0.008278,-0.008433,0.001328,-0.000220,0.010978
IRIS_Bcell-Memory_IgM,0.010564,-0.011524,0.078050,0.003501,-0.001604,-0.010693,-0.001642,-0.023532,-0.008902,0.003749,...,0.000308,-0.008006,-0.004237,0.002410,-0.004284,0.002100,0.011844,-0.009851,-0.002701,0.008679
IRIS_Bcell-naive,-0.001663,-0.009031,0.040280,0.054591,0.007393,0.004368,0.005322,-0.006171,-0.002761,0.017653,...,0.034196,0.002559,-0.003320,0.007691,0.019290,0.020934,0.004355,0.007455,0.004599,0.042783
IRIS_CD4Tcell-N0,0.000519,-0.001797,0.003385,0.005576,-0.005333,0.105796,-0.003038,0.008361,-0.005646,0.007226,...,0.013043,0.004532,0.006059,0.010275,0.020938,0.000495,-0.004533,0.015927,0.030064,0.011286
IRIS_CD4Tcell-Th1-restimulated12hour,0.004123,-0.008218,-0.009358,-0.007941,0.002393,0.017364,0.005469,0.000268,0.008350,0.009203,...,-0.004109,-0.004983,0.017694,-0.005510,0.011987,-0.011343,0.006783,0.007513,0.014512,0.002472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.016179,0.018159,0.031747,0.013597,-0.007673,-0.006613,-0.010845,0.019176,0.011361,0.017246,...,-0.009011,0.010139,0.002630,0.003714,-0.002336,0.005775,0.017198,0.001458,0.002374,0.009627
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,-0.003251,-0.014309,-0.022761,0.009124,-0.018531,-0.003891,-0.006762,-0.019643,-0.003727,-0.001721,...,-0.003915,-0.010865,-0.009200,-0.007061,-0.001459,-0.009086,-0.018587,-0.008488,-0.003278,-0.001296
PID_BCR_5PATHWAY,0.010677,0.016365,0.028507,0.025466,0.007074,0.009614,0.022995,0.001520,0.007916,0.005378,...,0.003617,-0.001084,0.000076,0.004518,0.005020,0.002713,0.005511,0.000871,0.001874,0.010344


In [6]:
Ur.rank(axis=0, ascending=False)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,319.0,360.0,538.0,326.0,93.0,431.0,194.0,278.0,253.0,569.0,...,8.0,203.0,400.0,570.0,498.0,558.0,510.0,334.0,367.0,172.0
IRIS_Bcell-Memory_IgM,213.0,491.0,1.0,297.0,380.0,496.0,384.0,579.0,495.0,280.0,...,354.0,543.0,465.0,270.0,462.0,289.0,97.0,556.0,426.0,206.0
IRIS_Bcell-naive,391.0,464.0,17.0,4.0,214.0,272.0,284.0,434.0,398.0,74.0,...,6.0,275.0,448.0,139.0,44.0,8.0,255.0,166.0,251.0,7.0
IRIS_CD4Tcell-N0,351.0,376.0,304.0,270.0,438.0,1.0,395.0,206.0,454.0,211.0,...,78.0,219.0,218.0,94.0,37.0,337.0,438.0,61.0,28.0,165.0
IRIS_CD4Tcell-Th1-restimulated12hour,299.0,450.0,467.0,466.0,313.0,98.0,282.0,332.0,179.0,174.0,...,472.0,492.0,65.0,503.0,108.0,582.0,196.0,165.0,106.0,332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,139.0,110.0,36.0,166.0,469.0,442.0,492.0,95.0,145.0,77.0,...,559.0,103.0,306.0,231.0,424.0,158.0,45.0,327.0,300.0,192.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,409.0,517.0,571.0,227.0,562.0,412.0,440.0,566.0,422.0,386.0,...,468.0,574.0,528.0,531.0,407.0,569.0,580.0,541.0,440.0,399.0
PID_BCR_5PATHWAY,211.0,123.0,49.0,61.0,217.0,186.0,91.0,309.0,190.0,243.0,...,271.0,399.0,367.0,209.0,237.0,260.0,221.0,346.0,317.0,182.0


In [7]:
Ur = Ur.rank(axis=0, ascending=False)

In [8]:
Ur

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,319.0,360.0,538.0,326.0,93.0,431.0,194.0,278.0,253.0,569.0,...,8.0,203.0,400.0,570.0,498.0,558.0,510.0,334.0,367.0,172.0
IRIS_Bcell-Memory_IgM,213.0,491.0,1.0,297.0,380.0,496.0,384.0,579.0,495.0,280.0,...,354.0,543.0,465.0,270.0,462.0,289.0,97.0,556.0,426.0,206.0
IRIS_Bcell-naive,391.0,464.0,17.0,4.0,214.0,272.0,284.0,434.0,398.0,74.0,...,6.0,275.0,448.0,139.0,44.0,8.0,255.0,166.0,251.0,7.0
IRIS_CD4Tcell-N0,351.0,376.0,304.0,270.0,438.0,1.0,395.0,206.0,454.0,211.0,...,78.0,219.0,218.0,94.0,37.0,337.0,438.0,61.0,28.0,165.0
IRIS_CD4Tcell-Th1-restimulated12hour,299.0,450.0,467.0,466.0,313.0,98.0,282.0,332.0,179.0,174.0,...,472.0,492.0,65.0,503.0,108.0,582.0,196.0,165.0,106.0,332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,139.0,110.0,36.0,166.0,469.0,442.0,492.0,95.0,145.0,77.0,...,559.0,103.0,306.0,231.0,424.0,158.0,45.0,327.0,300.0,192.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,409.0,517.0,571.0,227.0,562.0,412.0,440.0,566.0,422.0,386.0,...,468.0,574.0,528.0,531.0,407.0,569.0,580.0,541.0,440.0,399.0
PID_BCR_5PATHWAY,211.0,123.0,49.0,61.0,217.0,186.0,91.0,309.0,190.0,243.0,...,271.0,399.0,367.0,209.0,237.0,260.0,221.0,346.0,317.0,182.0


In [52]:
Ur_expected = pd.read_csv(data_dir / "Ur.csv.gz")

In [53]:
Ur_expected

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,319,360,538,326,93,431,194,278,253,569,...,8,203,400,570,498,558,510,334,367,172
1,213,491,1,297,380,496,384,579,495,280,...,354,543,465,270,462,289,97,556,426,206
2,391,464,17,4,214,272,284,434,398,74,...,6,275,448,139,44,8,255,166,251,7
3,351,376,304,270,438,1,395,206,454,211,...,78,219,218,94,37,337,438,61,28,165
4,299,450,467,466,313,98,282,332,179,174,...,472,492,65,503,108,582,196,165,106,332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,139,110,36,166,469,442,492,95,145,77,...,559,103,306,231,424,158,45,327,300,192
602,409,517,571,227,562,412,440,566,422,386,...,468,574,528,531,407,569,580,541,440,399
603,211,123,49,61,217,186,91,309,190,243,...,271,399,367,209,237,260,221,346,317,182
604,40,511,353,96,219,49,29,217,409,296,...,177,211,42,181,150,363,155,135,23,53


In [9]:
Urm = Ur.min(axis=1)

In [10]:
U = np.zeros(shape=(priorMat.shape[1], Z.shape[1]))

In [11]:
lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))
lambdas

array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919e-03, 1.32678043e-03, 1.17087962e-03, 1.03329764e-03,
       9.11881966e-04, 8.04733010e-04, 7.10174389e-04, 6.26726698e-04,
       5.53084370e-04, 4.88095244e-04, 4.30742541e-04, 3.80128958e-04,
       3.35462628e-04, 2.96044730e-04, 2.61258557e-04, 2.30559868e-04,
       2.03468369e-04, 1.79560205e-04, 1.58461325e-04, 1.39841629e-04,
       1.23409804e-04, 1.08908770e-04, 9.61116521e-05, 8.48182352e-05,
       7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
      

In [12]:
maxPath

10

In [50]:
np.where([Ur.iloc[:, 0] <= maxPath])

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([ 23,  62, 208, 230, 238, 274, 349, 354, 358, 593]))

In [54]:
np.where([Ur_expected.iloc[:, 0] <= maxPath])

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([ 23,  62, 208, 230, 238, 274, 349, 354, 358, 593]))

In [13]:
iip = np.where([Ur.iloc[:, 0] <= maxPath])[1]
iip

array([ 23,  62, 208, 230, 238, 274, 349, 354, 358, 593])

In [55]:
iip = np.where([Urm <= maxPath])[1]
iip

array([  0,   1,   2,   3,   5,   7,  10,  12,  13,  16,  18,  19,  20,
        21,  23,  24,  25,  26,  28,  30,  32,  33,  34,  35,  36,  40,
        41,  43,  45,  47,  48,  49,  53,  54,  56,  57,  60,  62,  67,
        74,  78,  88, 106, 109, 121, 123, 128, 134, 135, 137, 149, 153,
       154, 158, 164, 165, 166, 174, 175, 176, 183, 184, 186, 187, 193,
       194, 197, 199, 201, 203, 208, 224, 225, 227, 230, 232, 235, 238,
       246, 249, 252, 259, 262, 274, 281, 283, 290, 291, 324, 327, 328,
       333, 338, 345, 349, 354, 358, 361, 363, 364, 366, 372, 381, 386,
       390, 393, 404, 424, 426, 438, 441, 444, 446, 451, 453, 455, 463,
       467, 471, 479, 480, 485, 488, 496, 497, 503, 506, 508, 517, 518,
       530, 533, 534, 535, 536, 538, 547, 559, 560, 561, 564, 568, 569,
       574, 579, 581, 582, 591, 593, 597])

In [15]:
Urm[np.where([Urm <= maxPath])[1]]

pathway
IRIS_Bcell-Memory_IgG_IgA                8.0
IRIS_Bcell-Memory_IgM                    1.0
IRIS_Bcell-naive                         1.0
IRIS_CD4Tcell-N0                         1.0
IRIS_CD4Tcell-Th1-restimulated48hour     4.0
                                        ... 
PID_HIF2PATHWAY                          4.0
KEGG_N_GLYCAN_BIOSYNTHESIS               6.0
REACTOME_IL1_SIGNALING                  10.0
PID_AR_TF_PATHWAY                        5.0
KEGG_BASAL_TRANSCRIPTION_FACTORS         5.0
Length: 150, dtype: float64

In [16]:
from sklearn.linear_model import ElasticNet

In [17]:
len(penalty_factor)

606

In [18]:
penalty_factor = np.ones(606)

In [56]:
priorMat.iloc[:, iip].astype(np.float64).values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# from glmnet_python import glmnet
from glmnet import ElasticNet as gEN

In [22]:
# gres = glmnet(
#     y = Z[, i],
#     x = priorMat[, iip],
#     penalty.factor = penalty.factor[iip],
#     alpha = glm_alpha,
#     lower.limits = 0,
#     lambda = lambdas,
#     intercept = T,
#     standardize = F
# )

gres = glmnet(
    y=Z.iloc[:, 0].astype(np.float64).values,
    x=priorMat.iloc[:, iip].astype(np.float64).values,
    alpha=glm_alpha,
    penalty_factor=penalty_factor[iip],
    # lower_limits = 0,
    nlambda=lambdas,
    # intercept = True,
    standardize=False,
)

ElasticNet(alphas=lambdas)

<class 'OSError'>: /usr/lib/x86_64-linux-gnu/libgfortran.so.3: version `GFORTRAN_1.0' not found (required by /home/vscode/.cache/pypoetry/virtualenvs/pyplier-3Y0kqMGe-py3.8/lib/python3.8/site-packages/glmnet_python/GLMnet.so)

In [79]:
gres = gEN(
    lambda_path=lambdas,
    lower_limits=0,
    standardize=False,
    fit_intercept=True,
    alpha=glm_alpha,
    max_features=150,
)

In [80]:
len(penalty_factor[iip])

150

In [81]:
iip

array([  0,   1,   2,   3,   5,   7,  10,  12,  13,  16,  18,  19,  20,
        21,  23,  24,  25,  26,  28,  30,  32,  33,  34,  35,  36,  40,
        41,  43,  45,  47,  48,  49,  53,  54,  56,  57,  60,  62,  67,
        74,  78,  88, 106, 109, 121, 123, 128, 134, 135, 137, 149, 153,
       154, 158, 164, 165, 166, 174, 175, 176, 183, 184, 186, 187, 193,
       194, 197, 199, 201, 203, 208, 224, 225, 227, 230, 232, 235, 238,
       246, 249, 252, 259, 262, 274, 281, 283, 290, 291, 324, 327, 328,
       333, 338, 345, 349, 354, 358, 361, 363, 364, 366, 372, 381, 386,
       390, 393, 404, 424, 426, 438, 441, 444, 446, 451, 453, 455, 463,
       467, 471, 479, 480, 485, 488, 496, 497, 503, 506, 508, 517, 518,
       530, 533, 534, 535, 536, 538, 547, 559, 560, 561, 564, 568, 569,
       574, 579, 581, 582, 591, 593, 597])

In [82]:
gres.fit(
    y=Z.iloc[:, 0].astype(np.float64).values,
    X=priorMat.iloc[:, iip].astype(np.float64).values,
    relative_penalties=penalty_factor[iip],
)

ElasticNet(alpha=0.9,
           lambda_path=array([1.83156389e-02, 1.61634946e-02, 1.42642339e-02, 1.25881422e-02,
       1.11089965e-02, 9.80365504e-03, 8.65169520e-03, 7.63509422e-03,
       6.73794700e-03, 5.94621736e-03, 5.24751840e-03, 4.63091873e-03,
       4.08677144e-03, 3.60656314e-03, 3.18278080e-03, 2.80879419e-03,
       2.47875218e-03, 2.18749112e-03, 1.93045414e-03, 1.70361980e-03,
       1.50343919...
       7.48518299e-05, 6.60565080e-05, 5.82946637e-05, 5.14448602e-05,
       4.53999298e-05, 4.00652974e-05, 3.53575009e-05, 3.12028850e-05,
       2.75364493e-05, 2.43008313e-05, 2.14454083e-05, 1.89255064e-05,
       1.67017008e-05, 1.47391992e-05, 1.30072977e-05, 1.14788999e-05,
       1.01300936e-05, 8.93977622e-06, 7.88932483e-06, 6.96230472e-06,
       6.14421235e-06]),
           lower_limits=0, max_features=150, standardize=False)

In [86]:
beta = pd.DataFrame(gres.coef_path_)
beta

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010958,0.011528,0.012031,0.012476,0.012870,0.013220,0.013527,0.013764,0.013989,0.014187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008634,0.009159,0.009615,0.010017,0.010373,0.010689,0.010959,0.011207,0.011427,0.011621
146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008142,0.008475,0.008766,0.009020,0.009245,0.009444,0.009622,0.009777,0.009914,0.010036
148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054240,0.054440,0.054612,0.054760,0.054888,0.054994,0.055096,0.055180,0.055253,0.055317


In [87]:
np.count_nonzero(beta, axis=0)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,  3,  5,
        5,  8, 10, 13, 20, 24, 25, 27, 30, 32, 34, 35, 40, 42, 44, 44, 45,
       45, 45, 46, 47, 48, 48, 49, 49, 50, 50, 51, 51, 51, 51])

In [88]:
beta.sum(axis=0)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
        ...   
60    0.686709
61    0.693850
62    0.700257
63    0.705932
64    0.710943
Length: 65, dtype: float64

In [89]:
beta_expected = pd.read_csv(data_dir / "beta.csv")

In [90]:
beta_expected

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64
0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,0,0,0,0,0,0,0,0,0,...,0.010958,0.011528,0.012031,0.012476,0.012870,0.013220,0.013527,0.013764,0.013989,0.014187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0,0,0,0,0,0,0,0,0,0,...,0.008634,0.009159,0.009615,0.010017,0.010373,0.010689,0.010959,0.011207,0.011427,0.011621
146,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
147,0,0,0,0,0,0,0,0,0,0,...,0.008142,0.008475,0.008766,0.009020,0.009245,0.009444,0.009622,0.009777,0.009914,0.010036
148,0,0,0,0,0,0,0,0,0,0,...,0.054240,0.054440,0.054612,0.054760,0.054888,0.054994,0.055096,0.055180,0.055253,0.055317


In [109]:
np.full((len(lambdas), Z.shape[1]), np.nan)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [120]:
np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,  3,  5,
        5,  8, 10, 13, 20, 24, 25, 27, 30, 32, 34, 35, 40, 42, 44, 44, 45,
       45, 45, 46, 47, 48, 48, 49, 49, 50, 50, 51, 51, 51, 51])

In [189]:
from copy import deepcopy

In [190]:
lambdas = np.exp(np.arange(start=-4, stop=-12.125, step=-0.125))
results = dict()
lMat = np.full((len(lambdas), Z.shape[1]), np.nan)

for i in range(Z.shape[1]):
    if pathwaySelection == "fast":
        iip = np.where([Ur.iloc[:, i] <= maxPath])[1]
    else:
        iip = np.where([Urm <= maxPath])[1]

    gres = gEN(
        lambda_path=lambdas,
        lower_limits=0,
        standardize=False,
        fit_intercept=True,
        alpha=glm_alpha,
        max_features=150,
    )

    gres.fit(
        y=Z.iloc[:, i].astype(np.float64).values,
        X=priorMat.iloc[:, iip].astype(np.float64).values,
        relative_penalties=penalty_factor[iip],
    )
    gres.iip = iip
    lMat[:, i] = np.sum(np.where(gres.coef_path_ > 0, 1, 0), axis=0)
    results[i] = deepcopy(gres)
    del gres

In [100]:
expected_lMat = pd.read_csv(data_dir / "lMat.csv.gz")

In [225]:
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)

In [226]:
pd.DataFrame(lMat)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,50.0,46.0,53.0,54.0,58.0,48.0,47.0,55.0,43.0,44.0,...,59.0,57.0,55.0,55.0,56.0,51.0,46.0,64.0,58.0,70.0
61,51.0,46.0,53.0,55.0,58.0,48.0,47.0,55.0,43.0,44.0,...,60.0,58.0,56.0,55.0,56.0,52.0,47.0,65.0,59.0,71.0
62,51.0,47.0,53.0,55.0,59.0,49.0,48.0,55.0,43.0,44.0,...,61.0,59.0,56.0,56.0,57.0,53.0,48.0,67.0,59.0,71.0
63,51.0,47.0,53.0,55.0,59.0,49.0,48.0,56.0,44.0,44.0,...,62.0,59.0,57.0,56.0,58.0,54.0,49.0,67.0,59.0,71.0


In [227]:
expected_lMat

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,50,46,53,54,58,48,47,55,43,44,...,59,57,55,55,56,51,46,64,58,70
61,51,46,53,55,58,48,47,55,43,44,...,60,58,56,55,56,52,47,65,59,71
62,51,47,53,55,59,49,48,55,43,44,...,61,59,56,56,57,53,48,67,59,71
63,51,47,53,55,59,49,48,56,44,44,...,62,59,57,56,58,54,49,67,59,71


In [215]:
np.mean(np.where(lMat > 0, 1, 0), axis=1)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03333333, 0.06666667, 0.06666667,
       0.1       , 0.1       , 0.1       , 0.1       , 0.1       ,
       0.13333333, 0.13333333, 0.16666667, 0.2       , 0.2       ,
       0.23333333, 0.3       , 0.43333333, 0.6       , 0.6       ,
       0.7       , 0.76666667, 0.76666667, 0.83333333, 0.83333333,
       0.86666667, 0.96666667, 0.96666667, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [216]:
fracs = np.mean(np.where(lMat > 0, 1, 0), axis=1)

In [217]:
target_frac

0.7

In [219]:
np.where((target_frac - fracs) == min(abs(target_frac - fracs)))[0][0]

25

In [220]:
iibest = np.where((target_frac - fracs) == min(abs(target_frac - fracs)))[0][0]

In [229]:
len(iip)

150

In [211]:
for i in range(Z.shape[1]):
    U[results[i].iip, i] = results[i].coef_path_[:, iibest]

In [243]:
delta = {i: results[i].coef_path_[:, iibest] for i in range(Z.shape[1])}

In [251]:
echo = pd.DataFrame.from_dict(delta)
echo.index = Urm[np.where([Urm <= maxPath])[1]].index

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.005948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated48hour,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_HIF2PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KEGG_N_GLYCAN_BIOSYNTHESIS,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_IL1_SIGNALING,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [252]:
echo

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.005948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated48hour,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_HIF2PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KEGG_N_GLYCAN_BIOSYNTHESIS,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_IL1_SIGNALING,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
echo.loc["IRIS_Monocyte-Day0", 1]

0.0020993936030829475

In [260]:
pd.DataFrame(index=Urm.index).merge(echo, on="pathway", how="left").fillna(0)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.005948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_BCR_5PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
priorMat.columns

Index(['IRIS_Bcell-Memory_IgG_IgA', 'IRIS_Bcell-Memory_IgM',
       'IRIS_Bcell-naive', 'IRIS_CD4Tcell-N0',
       'IRIS_CD4Tcell-Th1-restimulated12hour',
       'IRIS_CD4Tcell-Th1-restimulated48hour',
       'IRIS_CD4Tcell-Th2-restimulated12hour',
       'IRIS_CD4Tcell-Th2-restimulated48hour', 'IRIS_CD8Tcell-N0',
       'IRIS_DendriticCell-Control',
       ...
       'KEGG_GNRH_SIGNALING_PATHWAY', 'KEGG_BASAL_TRANSCRIPTION_FACTORS',
       'REACTOME_SYNTHESIS_OF_DNA', 'KEGG_HEMATOPOIETIC_CELL_LINEAGE',
       'KEGG_T_CELL_RECEPTOR_SIGNALING_PATHWAY', 'PID_IL4_2PATHWAY',
       'REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', 'PID_BCR_5PATHWAY',
       'PID_TELOMERASEPATHWAY', 'PID_PI3KPLCTRKPATHWAY'],
      dtype='object', length=606)

In [204]:
results[0].coef_path_[:, 26]

array([0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.010639, 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.   

In [223]:
i = 1
iibest = 25
gamma = pd.DataFrame(
    results[i].coef_path_[:, iibest], index=Urm[np.where([Urm <= maxPath])[1]].index
)
pd.set_option("display.max_rows", 20)
gamma

,0
pathway,
IRIS_Bcell-Memory_IgG_IgA,0.0
IRIS_Bcell-Memory_IgM,0.0
IRIS_Bcell-naive,0.0
IRIS_CD4Tcell-N0,0.0
IRIS_CD4Tcell-Th1-restimulated48hour,0.0
...,...
PID_HIF2PATHWAY,0.0
KEGG_N_GLYCAN_BIOSYNTHESIS,0.0
REACTOME_IL1_SIGNALING,0.0


In [224]:
gamma[gamma[0] > 0]

,0
pathway,
IRIS_Monocyte-Day0,0.002099
IRIS_Neutrophil-Resting,0.162559


In [232]:
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)
pd.DataFrame(results[i].coef_path_, index=Urm[np.where([Urm <= maxPath])[1]].index)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IRIS_Bcell-Memory_IgM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IRIS_Bcell-naive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IRIS_CD4Tcell-N0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IRIS_CD4Tcell-Th1-restimulated48hour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_HIF2PATHWAY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003678,0.004163,0.004593,0.004967,0.005292,0.005579,0.005834,0.006059,0.006258,0.006434
KEGG_N_GLYCAN_BIOSYNTHESIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
REACTOME_IL1_SIGNALING,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.039484,0.039872,0.040216,0.040522,0.040790,0.041028,0.041238,0.041424,0.041588,0.041733


In [233]:
pd.DataFrame(U, index=Ur.index)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
pathway,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.015445,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.000000,0.0,0.0,0.007154,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PID_BCR_5PATHWAY,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
gamma = pd.DataFrame(np.zeros((3, 3)))
gamma

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0


In [239]:
gamma.iloc[[0, 2], 0] = [4, 5]

In [240]:
gamma

,0,1,2
0,4.0,0.0,0.0
1,0.0,0.0,0.0
2,5.0,0.0,0.0


In [274]:
s1 = pd.Series({"a": 1, "b": 2, "c": 3})

In [275]:
s2 = pd.Series({"a": 1, "b": 2, "c": 3, "d": 4})

In [278]:
pd.concat([s1, s2], axis=1).fillna(0)

,0,1
a,1.0,1
b,2.0,2
c,3.0,3
d,0.0,4
